# Sustainable Energy Academy AI API

## Main

An ETL pipeline for the SEA AI API. The pipeline prepares an SDG 7 tabular dataset, chunks texts for RAG and adds node and edge data for the knowledge graph to LanceDB.

### Libraries

In [1]:
import os
from io import BytesIO
from datetime import datetime

import httpx
import pandas as pd
from dotenv import load_dotenv
from langchain_text_splitters import TokenTextSplitter
from tqdm import tqdm

from src import genai, database

load_dotenv()

VERSION = f"{datetime.today():v%y-%m-%d}"
STORAGE_OPTIONS = database.get_storage_options()

In [2]:
connection = await database.get_connection()
await connection.table_names()

['chunks', 'edges', 'nodes', 'sdg7']

In [3]:
embedder = genai.get_embedding_client()
assert len(embedder.embed_query("Hi")) == 1024

### SDG 7 Data

Data on SDG 7 indicators from [UNSD SDGs API](https://unstats.un.org/SDGAPI/swagger/).

In [4]:
response = httpx.get("https://unstats.un.org/SDGAPI/v1/sdg/GeoArea/List")
df_areas = pd.DataFrame(response.json())
print("Shape:", df_areas.shape)
display(df_areas.head())

Shape: (460, 2)


,geoAreaCode,geoAreaName
0,4,Afghanistan
1,248,Åland Islands
2,8,Albania
3,12,Algeria
4,16,American Samoa


In [5]:
response = httpx.post(
    "https://unstats.un.org/SDGAPI/v1/sdg/Goal/DataCSV",
    data={
        "goal": [7],
        "areaCodes": df_areas["geoAreaCode"].tolist(),
        "timePeriodStart": 2010,
        "timePeriodEnd": 2025,
    },
    timeout=30,
)
df_sdg7 = pd.read_csv(BytesIO(response.content), low_memory=False)
print("Shape:", df_sdg7.shape)
display(df_sdg7.head())

Shape: (71979, 22)


,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,LowerBound,BasePeriod,Source,GeoInfoUrl,FootNote,[Location],[Nature],[Reporting Type],[Type of renewable technology],[Units]
0,7.0,7.1,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,1.0,World,2010.0,81.47449,2010.0,...,77.90507,NaN,"Global Health Observatory (GHO), World Health ...",NaN,NaN,URBAN,N,G,NaN,PERCENT
1,7.0,7.1,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,1.0,World,2010.0,57.16206,2010.0,...,52.95014,NaN,"Global Health Observatory (GHO), World Health ...",NaN,NaN,ALLAREA,N,G,NaN,PERCENT
2,7.0,7.1,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,1.0,World,2010.0,30.40394,2010.0,...,26.68803,NaN,"Global Health Observatory (GHO), World Health ...",NaN,NaN,RURAL,N,G,NaN,PERCENT
3,7.0,7.1,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,1.0,World,2011.0,31.74079,2011.0,...,27.93722,NaN,"Global Health Observatory (GHO), World Health ...",NaN,NaN,RURAL,N,G,NaN,PERCENT
4,7.0,7.1,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,1.0,World,2011.0,58.34087,2011.0,...,54.28883,NaN,"Global Health Observatory (GHO), World Health ...",NaN,NaN,ALLAREA,N,G,NaN,PERCENT


In [6]:
to_rename = {
    # "Goal": "",
    # "Target": "",
    "Indicator": "indicator_id",
    "SeriesCode": "indicator_code",
    "SeriesDescription": "indicator_name",
    # "GeoAreaCode": "",
    "GeoAreaName": "area_name",
    "TimePeriod": "year",
    "Value": "value",
    # "Time_Detail": "",
    # "TimeCoverage": "",
    # "UpperBound": "",
    # "LowerBound": "",
    # "BasePeriod": "",
    "Source": "source",
    # "GeoInfoUrl": "",
    # "FootNote": "",
    "[Location]": "location",
    # "[Nature]": "",
    # "[Reporting Type]": "",
    "[Type of renewable technology]": "technology",
    "[Units]": "units",
}

df_sdg7 = df_sdg7.reindex(to_rename, axis=1).rename(to_rename, axis=1)
df_sdg7.dropna(axis=0, how="all", ignore_index=True, inplace=True)
df_sdg7["year"] = df_sdg7["year"].astype(int)
assert df_sdg7.drop(columns=["value"]).duplicated().sum() == 0
assert (
    df_sdg7.duplicated(
        subset=["indicator_id", "area_name", "year", "location", "technology"]
    ).sum()
    == 0
)
print("Shape:", df_sdg7.shape)
display(df_sdg7.head())

Shape: (71978, 10)


,indicator_id,indicator_code,indicator_name,area_name,year,value,source,location,technology,units
0,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,World,2010,81.47449,"Global Health Observatory (GHO), World Health ...",URBAN,NaN,PERCENT
1,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,World,2010,57.16206,"Global Health Observatory (GHO), World Health ...",ALLAREA,NaN,PERCENT
2,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,World,2010,30.40394,"Global Health Observatory (GHO), World Health ...",RURAL,NaN,PERCENT
3,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,World,2011,31.74079,"Global Health Observatory (GHO), World Health ...",RURAL,NaN,PERCENT
4,7.1.2,EG_EGY_CLEAN,Proportion of population with primary reliance...,World,2011,58.34087,"Global Health Observatory (GHO), World Health ...",ALLAREA,NaN,PERCENT


In [7]:
table = await connection.create_table("sdg7", data=df_sdg7, mode="overwrite")
await table.count_rows()

71978

### Publications

Relevant [UNDP Publications](https://www.undp.org/publications) and [SDG 7 reports](https://trackingsdg7.esmap.org/downloads).

In [ ]:
df_corpus = pd.read_parquet(
    "abfs://datasets/corpus-v25-06-27.parquet", storage_options=STORAGE_OPTIONS
)
print("Shape:", df_corpus.shape)
display(df_corpus.head())

Shape: (419, 6)


,title,year,language,url,summary,text
0,Ocean for Development,2025,English,https://www.undp.org/publications/ocean-develo...,The world’s ocean is a vital foundation for su...,\n\n United Nations Development Programme (UND...
1,The Potential Growth and Future Trends of Gree...,2025,English,https://www.undp.org/publications/potential-gr...,This report underscores the potential of Green...,2024\r\nTHE POTENTIAL GROWTH \r\nAND FUTURE TR...
2,Rising up for SIDS: UNDP's Strategy to Support...,2025,English,https://www.undp.org/publications/rising-sids-...,UNDP’s new “Rising Up for SIDS” Strategy sets ...,1\r\nRising up\r\nfor SIDS\r\nUNDP’s Strategy ...
3,Community-Based Resilience-Building: A UNDP Gu...,2025,English,https://www.undp.org/publications/community-ba...,This Guidance Note complements UNDP's Crisis O...,1\r\nCommunity-Based \r\nResilience-Building:\...
4,The NDC Insights Series,2025,English,https://www.undp.org/publications/ndc-insights...,"The NDC Insights Series, a flagship knowledge ...",NDC Insights\r\nSeries\r\nIssue No. 1\r\nUNITE...


In [ ]:
texts = df_corpus["text"].str.replace(r"\s+", " ", regex=True).tolist()
metadatas = df_corpus.drop(columns=["text"]).to_dict(orient="records")

# first split into larger chunks
text_splitter = TokenTextSplitter(
    model_name=os.environ["AZURE_OPENAI_EMBED_MODEL"],
    chunk_size=768,
    chunk_overlap=768 // 4,  # 25% overlap
)
documents = text_splitter.create_documents(texts=texts, metadatas=metadatas)
print("Document count:", len(documents))

# add smaller chunks
text_splitter = TokenTextSplitter(
    model_name=os.environ["AZURE_OPENAI_EMBED_MODEL"],
    chunk_size=256,
    chunk_overlap=256 // 4,  # 25% overlap
)
documents.extend(text_splitter.create_documents(texts=texts, metadatas=metadatas))
print("Document count:", len(documents))

Document count: 22799
Document count: 91064


In [8]:
df_chunks = pd.DataFrame(
    [document.metadata | {"content": document.page_content} for document in documents]
)
df_chunks.sort_values(
    ["year", "title"], ascending=[False, True], ignore_index=True, inplace=True
)
df_chunks.drop_duplicates("content", keep="first", ignore_index=True, inplace=True)
print("Shape:", df_chunks.shape)
display(df_chunks.head())

Shape: (89852, 6)


,title,year,language,url,summary,content
0,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,TRACKING SDG7 2025 THE ENERGY PROGRESS REPORT ...
1,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,opinions expressed in the adaptation are the ...
2,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,SDG 7 indicator can be accessed at no charge ...
3,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,Clean Cooking Alliance • Denmark (Ministry of...
4,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,ESMAP–World Bank) oversaw the development of t...


In [9]:
# filter out text with more than 25% of numbers
mask = (
    df_chunks["content"].str.count(r"(\b\d+[\.\,]?\d*\b)")
    / df_chunks["content"].str.count(" ")
) < 0.25
mask.sum()

np.int64(82705)

In [10]:
# too many numbers
print(df_chunks.loc[~mask, "content"].sample().str.wrap(100).item())

3% 0.0% 0.0% 0.0% 3.6 3.6 0.0 40.6 a Romania 3.4% 24.1% 23.7% 24.4% 16.0% 1.2% 0.1% 4.8% 1.8% 0.5%
0.1% 0.0% 0.0% 63.4 145.9 12.3 907.5 b Russian Federation 3.8% 3.3% 3.3% 3.5% 0.7% 0.0% 0.0% 2.8%
0.0% 0.0% 0.0% 0.0% 0.0% 406.5 111.3 50.7 16232.5 b Rwanda 80.1% 90.7% 86.7% 86.0% 84.8% 0.0% 0.0%
1.1% 0.0% 0.1% 0.


In [11]:
print("Shape before:", df_chunks.shape)
df_chunks = df_chunks.loc[mask].reset_index(drop=True)
print("Shape after:", df_chunks.shape)
display(df_chunks.head())

Shape before: (89852, 6)
Shape after: (82705, 6)


,title,year,language,url,summary,content
0,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,TRACKING SDG7 2025 THE ENERGY PROGRESS REPORT ...
1,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,opinions expressed in the adaptation are the ...
2,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,SDG 7 indicator can be accessed at no charge ...
3,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,Clean Cooking Alliance • Denmark (Ministry of...
4,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,ESMAP–World Bank) oversaw the development of t...


In [12]:
token_count = sum(
    len(x) for x in text_splitter._tokenizer.encode_batch(df_chunks["content"].tolist())
)
print(f"Token count: {token_count:,}\nPrice: ~${token_count / 1_000 * 0.00013:.2f}")

Token count: 31,750,073
Price: ~$4.13


In [16]:
batch_size = 128
embeddings = []
for i in tqdm(range(0, len(df_chunks), batch_size)):
    texts = df_chunks.iloc[i : i + batch_size]["content"].tolist()
    try:
        embeddings.extend(embedder.embed_documents(texts))
    except Exception as e:
        print(e)
        embeddings.extend([None] * len(texts))
        continue

print("Shape before:", df_chunks.shape)
df_chunks["vector"] = embeddings
print("Shape after:", df_chunks.shape)
display(df_chunks.head())

100%|██████████| 647/647 [37:35<00:00,  3.49s/it]

Shape before: (82705, 6)
Shape after: (82705, 7)


,title,year,language,url,summary,content,vector
0,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,TRACKING SDG7 2025 THE ENERGY PROGRESS REPORT ...,"[-0.003449213458225131, -0.010466005653142929,..."
1,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,opinions expressed in the adaptation are the ...,"[-0.006614877376705408, 0.01775367744266987, -..."
2,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,SDG 7 indicator can be accessed at no charge ...,"[0.005415826570242643, 0.007200463674962521, -..."
3,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,Clean Cooking Alliance • Denmark (Ministry of...,"[0.009274660609662533, 0.004549182485789061, -..."
4,2025 Tracking SDG7 Report,2025,English,https://trackingsdg7.esmap.org/downloads,Full Report for 2025 Tracking SDG7 Report,ESMAP–World Bank) oversaw the development of t...,"[-0.009559696540236473, 0.012020393274724483, ..."


In [ ]:
df_chunks.to_parquet(f"data/chunks-{VERSION}.parquet", compression="gzip")

In [ ]:
table = await connection.create_table("chunks", data=df_chunks)
await table.count_rows()

82705

### Knowledge Graph

Node and edge data for the knowledge graph extracted from relevant publications. 

**Nodes**

In [4]:
df_nodes = pd.read_parquet(
    "abfs://datasets/nodes-v25-09-25.parquet", storage_options=STORAGE_OPTIONS
)
print("Shape:", df_nodes.shape)
display(df_nodes.head())

Shape: (22591, 3)


,name,description,weight
0,1.5° SCENARIO,IRENA’s energy transition pathway aiming to li...,1.000000
1,1.5°C ALIGNMENT,Climate target ensuring energy policies contri...,1.000000
2,1.5°C SCENARIO,The 1.5°C SCENARIO is an IRENA energy transiti...,3.302585
3,1.5°C SCENARIO (1.5-S),IRENA scenario outlining energy transition pat...,1.000000
4,100 MEGAWATT CAPACITY,Planned cooling system output power indicating...,1.000000


In [5]:
df_nodes["vector"] = embedder.embed_documents(df_nodes["name"].tolist())
print("Shape:", df_nodes.shape)
display(df_nodes.head())

Shape: (22591, 4)


,name,description,weight,vector
0,1.5° SCENARIO,IRENA’s energy transition pathway aiming to li...,1.000000,"[-0.02050161547958851, 0.004379780497401953, -..."
1,1.5°C ALIGNMENT,Climate target ensuring energy policies contri...,1.000000,"[-0.019135311245918274, -0.009119171649217606,..."
2,1.5°C SCENARIO,The 1.5°C SCENARIO is an IRENA energy transiti...,3.302585,"[-0.027025390416383743, 0.00241677975282073, -..."
3,1.5°C SCENARIO (1.5-S),IRENA scenario outlining energy transition pat...,1.000000,"[-0.01074069831520319, 0.016629312187433243, -..."
4,100 MEGAWATT CAPACITY,Planned cooling system output power indicating...,1.000000,"[-0.052859097719192505, 0.005553792230784893, ..."


In [6]:
df_nodes.to_parquet(f"data/nodes-{VERSION}.parquet")

In [7]:
table = await connection.create_table("nodes", data=df_nodes, mode="overwrite")
await table.count_rows()

22591

**Edges**

In [8]:
df_edges = pd.read_parquet(
    "abfs://datasets/edges-v25-09-25.parquet", storage_options=STORAGE_OPTIONS
)
print("Shape:", df_edges.shape)
display(df_edges.head())

Shape: (50878, 4)


,subject,object,description,weight
0,1.5°C SCENARIO,BIOENERGY WITH CCS,Combines bioenergy with CCS for negative emiss...,2.945910
1,1.5°C SCENARIO,CARBON CAPTURE AND STORAGE (CCS),Integrates CCS technologies to reduce carbon e...,2.945910
2,1.5°C SCENARIO,CLEAN ENERGY DEPLOYMENT,Rapid deployment of clean energy is a key miti...,3.197225
3,1.5°C SCENARIO,CLEAN HYDROGEN DEPLOYMENT,Encourages deployment of clean hydrogen as alt...,2.945910
4,1.5°C SCENARIO,DIRECT RENEWABLE ENERGY USE,Promotes increased direct use of renewable ene...,3.079442


In [9]:
print("Shape before:", df_edges.shape)
df_edges.insert(1, "predicate", "relates_to")
print("Shape after:", df_edges.shape)
display(df_edges.head())

Shape before: (50878, 4)
Shape after: (50878, 5)


,subject,predicate,object,description,weight
0,1.5°C SCENARIO,relates_to,BIOENERGY WITH CCS,Combines bioenergy with CCS for negative emiss...,2.945910
1,1.5°C SCENARIO,relates_to,CARBON CAPTURE AND STORAGE (CCS),Integrates CCS technologies to reduce carbon e...,2.945910
2,1.5°C SCENARIO,relates_to,CLEAN ENERGY DEPLOYMENT,Rapid deployment of clean energy is a key miti...,3.197225
3,1.5°C SCENARIO,relates_to,CLEAN HYDROGEN DEPLOYMENT,Encourages deployment of clean hydrogen as alt...,2.945910
4,1.5°C SCENARIO,relates_to,DIRECT RENEWABLE ENERGY USE,Promotes increased direct use of renewable ene...,3.079442


In [10]:
df_edges.to_parquet(f"data/edges-{VERSION}.parquet")

In [11]:
table = await connection.create_table("edges", data=df_edges, mode="overwrite")
await table.count_rows()

50878